In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib
from matplotlib import pyplot as plt

In [ ]:
class identityBlock(tf.keras.Model):
  def __init__(self, filters, kernelSize):
    super(identityBlock, self).__init__(name='')
    self.conv1 = tf.keras.layers.Conv2D(filters,kernelSize, padding='same')
    self.bn1 = tf.keras.layers.BatchNormalization()

    self.act = tf.keras.layers.Activation('relu')

    self.conv2 = tf.keras.layers.Conv2D(filters, kernelSize, padding='same')
    self.bn2 = tf.keras.layers.BatchNormalization()

    self.add = tf.keras.layers.Add()

  def call(self, input_s):
    x = self.conv1(input_s)
    x = self.bn1(x)
    x = self.act(x)
    x = self.conv2(x)
    x = self.bn2(x)
    x = self.add([input_s ,x])
    x = self.act(x)
    return x

In [ ]:
class miniResNet(tf.keras.Model):
  def __init__(self, classNum):
    super(miniResNet, self).__init__()
    self.mainConv  = tf.keras.layers.Conv2D(64, 7, padding='same')
    self.mainBN = tf.keras.layers.BatchNormalization()
    self.maxPool = tf.keras.layers.MaxPool2D((3,3))
    self.ib1 = identityBlock(64, 3)
    self.ib2 = identityBlock(64, 3)
    self.gap = tf.keras.layers.GlobalAveragePooling2D()
    self.classifier = tf.keras.layers.Dense(classNum, activation = 'softmax')
    self.act = tf.keras.layers.Activation('relu')

  def call(self, input__s):
    x = self.mainConv(input__s)
    x = self.mainBN(x)
    x = self.act(x)
    x = self.maxPool(x)
    x = self.ib1(x)
    x = self.ib2(x)
    x = self.gap(x)
    x = self.classifier(x)
    return x


In [ ]:
# def prepro(features):
#   return tf.cast(features['image'], tf.float32)/255, features['label']

In [ ]:
# mnist = tfds.load('mnist', split=tfds.Split.TRAIN)
# mnist = mnist.map(prepro).batch(32)

In [ ]:
mnist = tf.keras.datasets.mnist
(trainImage, trainLabel), (testImage, testLabel) = mnist.load_data()

In [ ]:
print(trainImage.shape,
      trainLabel.shape,
      testImage.shape,
      testLabel.shape)

(60000, 28, 28) (60000,) (10000, 28, 28) (10000,)


In [ ]:
trainImage = trainImage / 255
testImage = testImage /255
trainImage =trainImage.reshape(60000, 28, 28, 1)
testImage = testImage.reshape(10000, 28, 28, 1)
print(trainImage.shape,
      testImage.shape)

(60000, 28, 28, 1) (10000, 28, 28, 1)


In [ ]:
myModel = miniResNet(10)
myModel.compile(optimizer='adam',
                loss = 'sparse_categorical_crossentropy',
                metrics = ['accuracy'])
myModel.fit(trainImage, trainLabel, batch_size=32, epochs = 1)

1875/1875 [==============================] - 23s 6ms/step - loss: 0.1285 - accuracy: 0.9650
